In [1]:
import os

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import Angle
from astroquery.vizier import Vizier

import pprint

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.6f}'.format)

In [2]:
def query_variable_star(star_name):
    v = Vizier(columns=['*'])
    v.ROW_LIMIT = 5  # Limit the number of rows to keep the output manageable

    # Query the VSX (Variable Star Index) catalog
    result = v.query_object(star_name, catalog=["B/vsx/vsx"])

    # Check if results were returned
    if result and len(result) > 0:
        vsx_table = result[0]  # Assuming the star is found in the first returned table
        
        name = vsx_table['Name'][0]
        
        ra_angle = Angle(vsx_table['RAJ2000'][0]/15.0, u.hour)
        ra = ra_angle.to_string(unit=u.hour, sep=' ', precision=2, pad=True)
        
        dec_angle = Angle(vsx_table['DEJ2000'][0], u.deg)
        dec = dec_angle.to_string(unit=u.degree, sep=' ', precision=2, alwayssign=True, pad=True)        
        
        period = vsx_table['Period'][0] if 'Period' in vsx_table.colnames else "N/A"
        mag_max = vsx_table['max'][0] if 'max' in vsx_table.colnames else "N/A"
        mag_min = vsx_table['min'][0] if 'min' in vsx_table.colnames else "N/A"

        var_type = vsx_table['Type'][0] if 'Type' in vsx_table.colnames else "N/A"
        spectral_type = vsx_table['Sp'][0] if 'Sp' in vsx_table.colnames else "N/A"

        # Convert the table to a dictionary with custom key names
        custom_dict = {
            'name': name,
            'ra': ra,
            'dec': dec,
            'period': period,
            'mag_max': mag_max,
            'mag_min': mag_min,
            'var_type': var_type,
            'spectral_type': spectral_type
        }

        return custom_dict

    else:
        print("No results found for", star_name)
        return {}
    
import numpy as np
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy import units as u

def calculate_airmass(ra, dec, obs_time, lat, lon, height):
    # Create Time object for the observation time
    time = Time(obs_time)

    # Create EarthLocation object for the observatory
    location = EarthLocation(lat=lat*u.deg, lon=lon*u.deg, height=height*u.m)

    # Create SkyCoord object for the celestial object
    celestial_obj = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs')

    # Create AltAz frame for the given time and location
    altaz_frame = AltAz(obstime=time, location=location)

    # Transform the celestial object coordinates to the AltAz frame
    celestial_altaz = celestial_obj.transform_to(altaz_frame)

    # Calculate the zenith distance
    zenith_distance = 90 * u.deg - celestial_altaz.alt

    # Calculate AIRMASS using the zenith distance
    if zenith_distance < 60 * u.deg:
        airmass = 1 / np.cos(zenith_distance)
    else:
        airmass = 1 / (np.cos(zenith_distance) + 0.50572 * (96.07995 - zenith_distance.to(u.deg).value)**(-1.6364))

    return airmass.value


def dms_to_decimal(degrees, minutes, seconds, direction):
    return Angle(f"{degrees}d{minutes}m{seconds}s{direction}").value

def dump_header(filename):
    with fits.open(filename) as hdul:
        for hdu in hdul:
            print("Header of HDU:", hdu.name)
            for key, value in hdu.header.items():
                print(f"{key}: {value}")
            print("\n")

def update_header(filename, values_to_add, keys_to_remove):
    with fits.open(filename, mode='update') as hdul:

        for key, value in values_to_add.items():
            hdul[0].header[key] = value

        for key in keys_to_remove:
            if key in hdul[0].header:
                del hdul[0].header[key]
        hdul.flush()

In [ ]:
star = 'T Crb'
directory = '/Users/jwatts/astrophotography/Photometry/TCrb/Light/working/'

obj = star
star_data = query_variable_star(obj)
objectra = star_data.get('ra', 0.0)
objectdec = star_data.get('dec', 0.0)

latitude = dms_to_decimal(19, 42, 29.97, 'N')
longitude = dms_to_decimal(155, 58, 45.07, 'W')
altitude = (1535.4 * u.imperial.ft).to(u.m).value

objectra_deg = Angle(objectra, u.hour).to(u.deg).value
objectdec_deg = Angle(objectdec, u.deg).value

print (f"RA: {objectra}  ({objectra_deg})")
print (f"Dec: {objectdec}  ({objectdec_deg})")
print("===================")
pprint.pprint(star_data)

In [ ]:
# Example usage:
obs_time = '2024-04-20T14:30:36.746820'
airmass = calculate_airmass(objectra_deg, objectdec_deg, obs_time, latitude, longitude, altitude)
print(f"AIRMASS: {airmass}")

In [ ]:
values_to_add = {
    'SITEELEV': (altitude, 'Site elevation in meters'),
    'SITELAT': (latitude, 'Site latitude in degrees'),
    'SITELONG': (longitude, 'Site longitude in degrees'),
    'OBJECT': (obj, 'Target object name'),
    'OBJCTRA': (objectra, '[hms J2000] Target right ascension'),
    'OBJCTDEC': (objectdec, '[dms +N J2000] Target declination')
}

keys_to_remove = ['RA', 'DEC']

for filename in os.listdir(directory):
    if filename.endswith('.fit'):
        fits_file = os.path.join(directory, filename)
        print(f"FITS FILE: {fits_file}")
        update_header(fits_file, values_to_add, keys_to_remove)
        dump_header(fits_file)


In [ ]:
values_to_add = {}
keys_to_remove = []

for filename in os.listdir(directory):
    if filename.endswith('.fit'):
        fits_file = os.path.join(directory, filename)
        print(f"FITS FILE: {fits_file}")
        update_header(fits_file, values_to_add, keys_to_remove)
        dump_header(fits_file)